Election records splink demonstration

In [1]:
import pandas as pd
from splink.duckdb.linker import DuckDBLinker

pd.options.display.max_rows = 1000
election_df = pd.read_csv("/project/output/transformed/election_results_table.csv")

In [16]:
election_df = election_df.head(1000)

In [17]:
from splink.duckdb.blocking_rule_library import block_on

settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        block_on(["first_name", "last_name", "year","month","state"]),
    ],
}
linker = DuckDBLinker(election_df, settings)

linker.profile_columns(
    ["first_name", "last_name", "state"], top_n=10, bottom_n=5
)

alt.VConcatChart(...)

In [19]:
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl

settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        block_on(["first_name", "last_name", "year","month","county"]),
    ],
    "comparisons": [
        ctl.name_comparison("first_name", term_frequency_adjustments=True),
        ctl.name_comparison("last_name", term_frequency_adjustments=True),
        cl.exact_match("year", term_frequency_adjustments=True),
        cl.exact_match("month",  term_frequency_adjustments=True),
        cl.exact_match("county",  term_frequency_adjustments=True),
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
    "max_iterations": 10,
    "em_convergence": 0.01
}

linker = DuckDBLinker(election_df, settings)

In [20]:
linker.estimate_probability_two_random_records_match(
    [
        "l.first_name = r.first_name and l.last_name = r.last_name and l.county = r.county and l.year = r.year and l.month = r.month",
    ],
    recall=0.6,
)

If this is truly the case then you do not need to run the linkage model.
However this is usually in error; expected rules to have recall of 60%. Consider revising rules as they may have an error.
Probability two random records match is estimated to be  0.
This means that amongst all possible pairwise record comparisons, one in Infinity are expected to match.  With 499,500 total possible comparisons, we expect a total of around 0.00 matching pairs


In [21]:
linker.estimate_u_using_random_sampling(max_pairs = 5e6)


----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - last_name (no m values are trained).
    - year (no m values are trained).
    - month (no m values are trained).
    - county (no m values are trained).


In [22]:
training_blocking_rule = block_on(["first_name","last_name"])
training_session_names = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."first_name" = r."first_name") AND (l."last_name" = r."last_name")

Parameter estimates will be made for the following comparison(s):
    - year
    - month
    - county

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - last_name



SplinkException: Error executing the following sql for table `__splink__m_u_counts`(__splink__m_u_counts_45ad164c2):
CREATE TABLE __splink__m_u_counts_45ad164c2 AS
(
  WITH __splink__df_comparison_vectors AS (
    SELECT
      *
    FROM __splink__df_comparison_vectors_f7d241f7d
  ), __splink__df_match_weight_parts AS (
    SELECT
      "unique_id_l",
      "unique_id_r",
      gamma_year,
      CASE
        WHEN gamma_year = -1
        THEN CAST(1.0 AS DOUBLE)
        WHEN gamma_year = 1
        THEN CAST(1.0477455337921533 AS DOUBLE)
        WHEN gamma_year = 0
        THEN CAST(0.5359557072040178 AS DOUBLE)
      END AS bf_year,
      CASE
        WHEN gamma_year = -1
        THEN CAST(1 AS DOUBLE)
        WHEN gamma_year = 1
        THEN (
          CASE
            WHEN NOT COALESCE("tf_year_l", "tf_year_r") IS NULL
            THEN POWER(
              CAST(0.9067087087087087 AS DOUBLE) / (
                CASE
                  WHEN COALESCE("tf_year_l", "tf_year_r") >= COALESCE

In [8]:
training_blocking_rule = block_on(["year","month"])
training_session_date = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."year" = r."year") AND (l."month" = r."month")

Parameter estimates will be made for the following comparison(s):
    - first_name
    - last_name
    - state

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - year
    - month

Iteration 1: Largest change in params was -0.0499 in the m_probability of state, level `All other comparisons`
Iteration 2: Largest change in params was 0.0101 in the m_probability of first_name, level `Damerau_levenshtein <= 1`
Iteration 3: Largest change in params was 0.00206 in the m_probability of first_name, level `Damerau_levenshtein <= 1`

EM converged after 3 iterations

Your model is fully trained. All comparisons have at least one estimate for their m and u values


In [9]:
linker.match_weights_chart()

alt.VConcatChart(...)

In [13]:
df_predict = linker.predict()
df_e = df_predict.as_pandas_dataframe(limit = 5)
df_e[["first_name_l","last_name_l"]]

,first_name_l,last_name_l
0,greg,albritton
1,greg,albritton
2,greg,albritton
3,greg,albritton
4,greg,albritton


In [ ]:
# district in ind_table -> the office they're running for
# try splink function 
# 

In [15]:
election_df[(election_df["first_name"] == "greg") & (election_df["last_name"] == "albritton")]

,Unnamed: 0,year,month,day,state,county,district_designation_ballot,district,district_number,geographic_post,...,vote,term,full_name,senate,party,district_seat_number,outcome,last_name,first_name,unique_id
715,5678,2014,11,4,AL,mobile,22,NaN,22,0,...,655,4.0,"albritton, greg",1,republican,1,w,albritton,greg,ca8befe5-7ffc-4563-8cf4-18adefe4f5ea
716,5679,2014,11,4,AL,choctaw,22,NaN,22,0,...,700,4.0,"albritton, greg",1,republican,1,w,albritton,greg,11fdbc1a-eeaa-4227-8feb-36633881de50
717,5680,2014,11,4,AL,conecuh,22,NaN,22,0,...,1003,4.0,"albritton, greg",1,republican,1,w,albritton,greg,518c68d3-d0d9-496a-902c-a3271b0d9320
718,5681,2014,11,4,AL,monroe,22,NaN,22,0,...,2179,4.0,"albritton, greg",1,republican,1,w,albritton,greg,532ce6cd-633f-490a-8500-60868f227477
719,5682,2014,11,4,AL,washington,22,NaN,22,0,...,2726,4.0,"albritton, greg",1,republican,1,w,albritton,greg,a5ff357f-5c45-4eed-97ed-c67ef2f23829
720,5683,2014,11,4,AL,clarke,22,NaN,22,0,...,3281,4.0,"albritton, greg",1,republican,1,w,albritton,greg,eb1ae14c-6826-4aa1-8fd3-bbbd0dcd6e6f
721,5684,2014,11,4,AL,escambia,22,NaN,22,0,...,5233,4.0,"albritton, greg",1,republican,1,w,albritton,greg,629766fd-365a-403f-a1f5-02914f3c0f37
722,5685,2014,11,4,AL,baldwin,22,NaN,22,0,...,7385,4.0,"albritton, greg",1,republican,1,w,albritton,greg,0b59008a-3c9e-4497-8482-c8b156d823d5


In [8]:
def splink_dedupe(df: pd.DataFrame, settings: dict, blocking: list) -> pd.DataFrame:
    """Use splink to deduplicate dataframe based on settings
    
    Configuration settings and blocking can be found in constants.py as
    individuals_settings, individuals_blocking, organizations_settings,
    organizations_blocking

    Uses the splink library which employs probabilistic matching for
    record linkage
    https://moj-analytical-services.github.io/splink/index.html

    Args:
        df: dataframe
        settings: configuration settings
            (based on splink documentation and dataframe columns)
        blocking: list of columns to block on for the table
            (cuts dataframe into parts based on columns labeled blocks)

    Returns:
        deduplicated version of initial dataframe with column 'matching_id'
        that holds list of matching unique_ids
    """
    # Initialize the linker object
    linker = DuckDBLinker(df, settings)

    # Estimate probability that two random records match
    linker.estimate_probability_two_random_records_match(
        blocking, recall=0.80
    )

    # Estimate the parameter u using random sampling
    linker.estimate_u_using_random_sampling(max_pairs=5e6)

    # Run expectation maximisation on each block
    for block in blocking:
        linker.estimate_parameters_using_expectation_maximisation(block)

    # Predict matches
    df_predict = linker.predict()

    # Cluster predictions and threshold
    clusters = linker.cluster_pairwise_predictions_at_threshold(
        df_predict, threshold_match_probability=0.7
    )
    clusters_df = clusters.as_pandas_dataframe()

    match_list_df = (
        clusters_df.groupby("cluster_id")["unique_id"].agg(list).reset_index()
    )
    match_list_df = match_list_df.rename(columns={"unique_id": "duplicated"})

    deduped_df = df.merge(
        match_list_df,
        left_on="unique_id",
        right_on="duplicated",
        how="left"
    )

    deduped_df["matching_id"] = deduped_df["cluster_id"]


    deduped_df = deduped_df.drop(columns=["duplicated", "cluster_id"])

    return deduped_df

In [11]:
splink_dedupe(election_df, settings, blocking )

Probability two random records match is estimated to be  4.13e-05.
This means that amongst all possible pairwise record comparisons, one in 24,186.40 are expected to match.  With 1,343,977,935 total possible comparisons, we expect a total of around 55,567.50 matching pairs
----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - last_name (no m values are trained).
    - year (no m values are trained).
    - month (no m values are trained).
    - state (no m values are trained).

----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.first_name = r.first_name and l.last_name = r.last_name and l.state = r.state and l.year = r.year and l.month = r.month

Parameter estimates will be made for the following comparison(s):

Parameter estimates cannot be made for the following compar

SplinkException: Error executing the following sql for table `__splink__m_u_counts`(__splink__m_u_counts_cae27a405):

        CREATE TABLE __splink__m_u_counts_cae27a405
        AS
        (WITH __splink__df_comparison_vectors as (select * from __splink__df_comparison_vectors_8e81f7a75), 
__splink__df_match_weight_parts as (
    select "unique_id_l","unique_id_r" 
    from __splink__df_comparison_vectors
    ), 
__splink__df_predict as (
    select
    log2(cast(1717.6657519007895 as float8) * ) as match_weight,
    CASE WHEN  THEN 1.0 ELSE (cast(1717.6657519007895 as float8) * )/(1+(cast(1717.6657519007895 as float8) * )) END as match_probability,
    "unique_id_l","unique_id_r" 
    from __splink__df_match_weight_parts
    
    order by 1
    ) 
    select 0 as comparison_vector_value,
           sum(match_probability * 1) /
               sum(1) as m_count,
           sum((1-match_probability) * 1) /
               sum(1) as u_count,
           '_probability_two_random_records_match